In [ ]:
import os
import re

import numpy as np
import pandas as pd

import gvar as gv
import lsqfit

from luescher_nd.database import utilities as ut
from luescher_nd.database.utilities import DATA_FOLDER

import seaborn as sns
import matplotlib.pylab as plt
import matplotlib

from luescher_nd.plotting import styles
from luescher_nd.plotting.styles import MARKERS

In [ ]:
A1_WIDTH = 6.0

In [ ]:
styles.setup(pgf=False)

In [ ]:
matplotlib.use("pgf")
sns.set(
    context="paper",
    style="ticks",
    font_scale=1/1.7,
    rc={
        # "mathtext.fontset": "cm",
        "pgf.rcfonts": False,
        "axes.unicode_minus": False,
        "font.serif": [],
        "font.sans-serif": [],
    },
)

In [ ]:
%load_ext blackcellmagic

In [ ]:
a_inv = 0.0
L = 1.0

In [ ]:
file_name = f"contact-fitted_a-inv={a_inv:+1.1f}_zeta=spherical_projector=a1g_n-eigs=200.sqlite"
print(file_name)
df = ut.read_table(
    os.path.join(DATA_FOLDER, file_name),
    zeta="spherical",
    round_digits=2,
    filter_poles=True,
    filter_by_nstates=False,
    filter_degeneracy=True,
).query("nlevel < 24 and n1d > 5")[
    ["n1d", "epsilon", "nstep", "L", "x", "nlevel", "mass", "y"]
]
df["L"] = df.L.round(7)
df.head()

def nstep_label(nstep) -> str:
    return "$" + (str(nstep) if nstep > 0 else r"\infty") + "$"

df["nstep_label"] = df.nstep.apply(nstep_label)

In [ ]:
tf = (
    df.loc[df.nlevel % 2 == 0]
    .query("x < 20 and y < 5 and y > 0 and nstep != 1 and x > 0 and L ==1")
    .sort_values(["epsilon"], ascending=True)
)

grid = sns.FacetGrid(
    data=tf,
    col="nstep",
    row="nlevel",
    sharex=True,
    sharey=True,
    legend_out=True,
    margin_titles=True,
    col_order=[2, 4, -1],
    row_order=np.sort(tf.nlevel.unique())[::-1],
    aspect=3,
    height=1.5,
)


grid.map(plt.plot, "epsilon", "y", ls="--", lw="0.5", marker=".", ms=1)
[plt.setp(ax.texts, text="") for ax in grid.axes.flat]

for ax in grid.axes.flat:
    ax.set_yscale("log")
    ax.set_xscale("log", basex=2)

grid.set_ylabels(r"")
grid.set_xlabels(r"")
grid.set_titles(
    row_template=r" "*10 + r"$E_{{{row_name}}}$" + " "*10,
    col_template=r"$n_{{\mathrm{{step}}}} = {col_name}$",
)

plt.subplots_adjust(wspace=0.2, hspace=0.2)

ratio = grid.fig.get_figheight() / grid.fig.get_figwidth()
grid.fig.set_figheight(ratio * A1_WIDTH)
grid.fig.set_figwidth(A1_WIDTH)

grid.fig.text(
    0.5,
    0.5,
    "Preliminary",
    fontsize=60,
    color="grey",
    ha="center",
    va="center",
    alpha=0.2,
    rotation=30,
    zorder=10,
)

grid.fig.text(
    -0.05,
    0.5,
    "$p \cot(\delta(p)) \, [\mathrm{fm}^{-1}]$",
    rotation=90,
    va="baseline",
    ha="center",
    fontsize=8,
    color="black",
)
grid.fig.text(
    0.5,
    -0.00,
    "$\epsilon [\mathrm{fm}]$",
    va="top",
    ha="center",
    fontsize=8,
    color="black",
)


grid.fig.set_dpi(300)

In [ ]:
grid.savefig("ere-continuum-extrapolation.pgf", bbox_axes="tight")